In [1]:
import os.path
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

from google.oauth2 import service_account
import pandas as pd
import os

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

In [2]:
def send_email(subject, message_text, recipient_email):
    """Sends an email using the Gmail API.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                # Change this to the .json verification file that you create in Google Cloud Console (0Auth2)
                'client_secret_171250538774-1hv52jle9rmtc3hpkfhttgs7sdr0ts8d.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)

    message = MIMEMultipart()
    message['to'] = recipient_email
    message['subject'] = subject
    msg = MIMEText(message_text)
    message.attach(msg)

    raw = base64.urlsafe_b64encode(message.as_bytes())
    raw = raw.decode()
    body = {'raw': raw}
    try:
        sent_message = (service.users().messages().send(userId="me", body=body)
                   .execute())
        print('Message Id: %s' % sent_message['id'])
        return sent_message
    except Exception as error:
        print(f'An error occurred: {error}')
        return None

In [3]:
def prepare_and_send_email(recipient_data):
    """
    Prepares and sends an email to the recipient based on provided data.
    
    Parameters:
    recipient_data (list): A list containing [First Name, Last Name, Email, Team Name].
    """
    first_name, last_name, email, team_name = recipient_data
    subject = "Congratulations and Welcome to the Team!"
    message_text = f"""
    Dear {first_name} {last_name},

    Congratulations on joining our team, {team_name}! We are thrilled to have you with us and look forward to achieving great things together.

    Please remember to check our team calendar for upcoming meetings and events. Staying informed will help us all stay on track and make the most out of our collaboration.

    Welcome aboard!

    Best regards,
    Konsing Ham Lopez
    Eco Car Team Leader
    ham.konsing@gmail.com
    """
    send_email(subject, message_text, email)

In [4]:
# Example usage, recipient_data[2] will receive an email
recipient_data = ["Ahmed", "Irtija", "ham.konsing@gmail.com", "Automation PM"]
prepare_and_send_email(recipient_data)

Message Id: 18dd3b1d0a876c67


In [5]:
def send_email_to_leader(new_member_data, leader_email):
    """
    Sends an email to the team leader informing them of a new team member.

    Args:
    new_member_data (list): A list containing [First Name, Last Name, Email, Team Name].
    leader_email (str): Email address of the team leader.
    """
    creds = None
    # Check for existing token.json for credentials
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Authenticate if no valid credentials
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_171250538774-1hv52jle9rmtc3hpkfhttgs7sdr0ts8d.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)

    first_name, last_name, email, team_name = new_member_data
    subject = f"New Team Member: {first_name} {last_name}"
    message_text = f"""
    Dear Team Leader,

    I'm pleased to inform you that {first_name} {last_name} ({email}) has joined our team, {team_name}. 

    Could you please update the team roster to reflect this new addition? Your attention to this matter is greatly appreciated.

    Best regards,
    Konsing
    Goat
    goat@gmail.com
    """

    message = MIMEMultipart()
    message['to'] = leader_email
    message['subject'] = subject
    msg = MIMEText(message_text)
    message.attach(msg)

    raw = base64.urlsafe_b64encode(message.as_bytes())
    raw = raw.decode()
    body = {'raw': raw}
    try:
        sent_message = service.users().messages().send(userId="me", body=body).execute()
        print('Message Id:', sent_message['id'])
    except Exception as error:
        print(f'An error occurred: {error}')

In [6]:
# Example usage, leader_email's string will be sent an email with the new_member_data
new_member_data = ["Ahmed", "Irtija", "airtija@ucdavis.edu", "Automation (PM)"]
leader_email = "ham.konsing@gmail.com"
send_email_to_leader(new_member_data, leader_email)

Message Id: 18dd3b1d30951523
